# Data manipulation verbs

An oft-repeated statistic is that 80% of data science work is data manipulation and preparation. The data manipulation verbs are the workhorses of this phase.

In [ ]:
import numpy as np
import pandas as pd

Load `epl_2020_2021.parquet` into a Pandas data frame (`df`).

This contains all the English Premier league matches during the 2020/2021 season.

In [ ]:
df = pd.read_parquet("epl_2020_2021.parquet")

View the contents of the `df` data frame.

In [ ]:
df

## Select

Selecting is choosing a subset of columns. Data sources often contain many irrelavant fields---often large text fields.

Reducing the data to contain just the column you need makes it easier to work with and is memory efficient.

Select the `HomeTeam`, `AwayTeam`, `FTHG` and `total_revenue`, `FTAG`, `B365H`, `B365D` and `B365A` columns from `df`. Save the result in `fulltime_df`.

In [ ]:
fulltime_df = df.loc[:, ["HomeTeam", "AwayTeam", "FTHG", "FTAG", "B365H", "B365D", "B365A"]]

Give the fields better names.

In [ ]:
fulltime_df.rename({
    "HomeTeam": "home_team", 
    "AwayTeam": "away_team", 
    "FTHG": "home_goals", 
    "FTAG": "away_goals",
    "B365H": "home_win_price",
    "B365D": "draw_price",
    "B365A": "away_win_price",
}, axis=1, inplace=True)

View the contents of `fulltime_df`.

In [ ]:
fulltime_df

We'll use `fulltime_df` in subsequent steps as it's easier to work with.

## Filter

Filtering allows us to select only the observation (rows) of interest.

Find all the matches in `fulltime_df` where Manchester City played Manchester United.

In [ ]:
fulltime_df[fulltime_df["home_team"].isin(("Man City", "Man United")) & fulltime_df["away_team"].isin(("Man City", "Man United"))]

## Sort

Sorting allows us to rearrange the order of the observations (rows). We can use this to examine outliers. 

It's also useful when presenting the data (in tabular or chart formats).

Sort `fulltime_df` in descending order of `home_win_price`.

In [ ]:
fulltime_df.sort_values("home_win_price", ascending=False)

## Mutate

Mutating allows us to modify or create new data---often entire columns.

Create a new `goal_difference` field in `fulltime_df`.

In [ ]:
fulltime_df["goal_difference"] = (fulltime_df["home_goals"] - fulltime_df["away_goals"]).abs()

View the contents of `fulltime_df`.

In [ ]:
fulltime_df

We can also mutate multiple columns using a custom function.

Create a `calculate_probability` function that takes a decimal betting price and returns an implied probability.

In [ ]:
def calculate_probability(price):
    return 1 / price

Create `home_win_probability`, `draw_probability` and `away_win_probability` columns in `fulltime_df`.

In [ ]:
fulltime_df[[
    "home_win_probability", 
    "draw_probability", 
    "away_win_probability"
]] = fulltime_df[[
    "home_win_price", 
    "draw_price", 
    "away_win_price"
]].apply(calculate_probability, result_type="expand")

View the (expanded) contents of `fulltime_df`.

In [ ]:
fulltime_df

Drop the implied probability columns from `fulltime_df`.

In [ ]:
fulltime_df.drop(["home_win_probability", "draw_probability", "away_win_probability"], axis=1, inplace=True)

We can also apply a custom function to multiple columns.

Define a `calculate_goal_difference` function.

In [ ]:
def calculate_goal_difference(home_goals, away_goals):
    return abs(home_goals - away_goals)

Apply `calculate_goal_difference` to `fulltime_df`.

In [ ]:
fulltime_df[["home_goals", "away_goals"]].apply(lambda x: calculate_goal_difference(*x), axis=1)

## Aggregate

Aggregating allows us to summarise across a number of rows, such as the entire data frame.

Calculate the maximum value of all the numeric fields in `fulltime_df`.

In [ ]:
fulltime_df.max(numeric_only = True)

Calculate the total of all the numeric fields in `fulltime_df`.

In [ ]:
fulltime_df.sum(numeric_only = True)

Calculate the median of all the numeric fields in `resources_df`.

In [ ]:
fulltime_df.median(numeric_only = True)

Or calculate them all at once.

In [ ]:
fulltime_df.loc[:, "home_goals":"goal_difference"].agg(["max", "sum", "median"]).T

## Group

Grouping allows us to split the data into subsets. It's often used in conjunction with aggregation.

Get the mean values for all columns in `fulltime_df` broken down (grouped) by `home_team`.

In [ ]:
fulltime_df.groupby("home_team").mean()

Calculate median and IQR grouped by home team.

In [ ]:
def iqr(values):
    quartile1, quartile3 = values.quantile([0.25, 0.75])
    
    return quartile3 - quartile1

In [ ]:
fulltime_df.groupby("home_team")[["home_goals", "away_goals"]].agg(["median", iqr])

## Takeaway

Data manipulation is at the core of data wranging. Mastering the data manipulation verbs will make it easier for you to explore you data and prepare it for downstream modelling and analysis.